# Appendices

Source code to replicate key results from the appendix

In [ ]:
# Libraries and imports
import sys
sys.path.append('../')

import os
import numpy as np
import matplotlib.pyplot as plt
from utils import helpers
import torchvision
from PIL import Image
from torchvision.models import resnet50
import timeit
import torch
from spectral_sobol.torch_explainer import WaveletSobol


## Number of designs

The number of designs corresponds to the number of deviations around the mean sequence. It is used to compute the variance of the Sobol indices. However, a high number of designs leads to an increased computational burden. 

We study the effect of the number of designs on the estimation of the Sobol indices

In [ ]:
# Set up 

device = 'cuda:2'
model = resnet50(pretrained = True).to(device)

classes = {
 'fox.png': 278,
 'snow_fox.png': 279,
 'polar_bear.png': 296,
 'leopard.png': 288,
 'fox1.jpg': 277,
 'fox2.jpg': 277,
 'sea_turtle.jpg': 33,
 'lynx.jpg': 287,
 'cat.jpg': 281,
 'otter.jpg': 360
}

image_names = list(classes.keys())

images = [Image.open('../assets/{}'.format(img_name)) for img_name in image_names]

# transforms
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preprocessing = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize,
])
 
x = torch.stack(
    [preprocessing(img) for img in images]
)

y = np.array([classes[img_name] for img_name in image_names])

# loop with a varying number of designs

variants = []
times = []

for i in range(5):

    print('Testing with i = {}'.format(i))
    wavelet = WaveletSobol(model, grid_size = 28, nb_design = 2 ** (i+1), batch_size = 128)

    start = timeit.default_timer()
    variants.append(wavelet(x,y))
    stop = timeit.default_timer()

    times.append(stop - start)

In [ ]:
# plots
plt.rcParams.update({'font.size': 15})


size = 224 # input size
levels = 3 # number of levels in the wavelet decomposition

indices = [3,4,7]

n_rows = len(indices)
n_cols = len(variants)

fig, ax = plt.subplots(n_rows, n_cols, figsize = (4 * n_cols, 4 * n_rows))

for j in range(n_cols):

    variant = variants[j] # retrieve the list of wcams
    
    for k, index in enumerate(indices) : # only plot some samples

        nb_design = 2 ** (j+1)

        title = 'Number of designs : {} \n Computation time (s) : {:0.2f}'.format(nb_design, times[j] / 10)
        
        ax[0,j].set_title(title)
        ax[k,j].imshow(variant[index], cmap = 'jet')
        ax[k,j].axis('off')

        helpers.add_lines(size, levels, ax[k,j])

fig.tight_layout()


plt.savefig('../figs/nb_design.pdf')
fig.tight_layout()
plt.show()

## Grid size and number of levels

Evaluate the effect of the grid size and the number of levels on the shape of the WCAM

## Consistency between Fourier-CAMs, WCAMs and existing literature

In [ ]:
# inputs by the user
n_samples = 100
data_dir = "../../data/ImageNet"
device = 'cuda:2'

In [ ]:
# set ups

# data a set of samples from IN validation set
samples = [s for s in os.listdir(data_dir) if s[-5:] == '.JPEG']
np.random.seed(42)
samples = np.random.choice(samples, n_samples) # restrict the list of samples

# load the labels dataframe
labels = helpers.format_dataframe(data_dir, filter = samples)

# transforms
preprocess = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224)
])

normalize = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

# load the images and generate the labels 
images = [preprocess(Image.open(os.path.join(data_dir, s)).convert('RGB')) for s in samples]
y = np.array([ # generate the labels
    labels[labels['name'] == s]['label'].values.item() for s in samples
])

# images passed to the model
x = torch.stack([
    normalize(im) for im in images
])

In [ ]:
# model set up
# model (and case)
# model zoo features : 
#               - RT : 'augmix', 'pixmix', 'sin' (highest accuracy on ImageNet-C), 
#               - AT : 'adv_free, fast_adv and adv,
#               - ST : standard training ('baseline')

models_dir = '../../models/spectral-attribution-baselines'
cases = ['baseline', 'augmix', 'pixmix', 'sin', 'adv_free', 'fast_adv', 'adv']

# load the model

models = []
for case in cases:
    
    if case == 'baseline':
        model = resnet50(pretrained = True).to(device).eval()
    elif case in ['augmix', 'pixmix', 'sin']:
        model = resnet50(pretrained = False) # model backbone #torch.load(os.path.join(models_dir, '{}.pth'.format(case))).eval()
        weights = torch.load(os.path.join(models_dir, '{}.pth.tar').format(case))
        model.load_state_dict(weights['state_dict'], strict = False)
        model.eval()
    elif case in ['adv_free', 'fast_adv', 'adv']:
        model = resnet50(pretrained = False) # model backbone #torch.load(os.path.join(models_dir, '{}.pth'.format(case))).eval()
        weights = torch.load(os.path.join(models_dir, "{}.pth".format(case)))
        model.load_state_dict(weights)
        model.eval()

    # we will loop over the models
    models.append(model)

### Consistency between Fourier-CAM and existing works

We leverage the Fourier-CAM to measure the average frequency importance over the selected batch of images. 

#### Circular masks

In [ ]:
# perturbation 
perturbation = 'circle' # other values are 'square' and 'grid' for the replication of 
                        # former paper's results 
grid_size = 14

results = {}

for case, model in zip(cases, models):
    print('Case ........... {}'.format(case))
    fourier = FourierSobol(model, grid_size = grid_size, nb_design = 16, batch_size=128, perturbation = perturbation)
    _ = fourier(x,y)
    results[case] = fourier.components


In [ ]:
# plots
# represent the importance of each frequency component in each case-
plt.rcParams.update({'font.size': 12})

# average the contributions per training mode
average_baseline = np.sum(results['baseline'], axis = 0) / n_samples
average_robust = np.sum(
    [
    np.sum(results[case], axis = 0) / n_samples for case in ['augmix', 'pixmix', 'sin']
    ], axis = 0) / 3

average_adversarial = np.sum(
    [
    np.sum(results[case], axis = 0) / n_samples for case in ['adv_free', 'fast_adv', "adv"]
    ], axis = 0) / 3

# average the contributions for each case
average_contributions = {
    'ST' : average_baseline,
    'RT' : average_robust,
    'AT' : average_adversarial
    }

# plots
for i, case in enumerate(average_contributions.keys()):

    average_contribution = average_contributions[case]

    offset = 0.33 * i

    plt.bar(np.array([*range(len(average_contribution))]) - 0.33 + offset , average_contribution, label = case, width = 0.2)


plt.legend()

# plot labels
plt.xlabel('Frequency component\n (The higher the higher the frequency)')
plt.ylabel('Importance (STI)')
plt.title('Importance of the frequency components (circular case)')
plt.xlim(-1,(grid_size // 2) + (grid_size // 4)) # resize as many circular components have no contribution
plt.xticks([*range(0, (grid_size // 2) + (grid_size // 4))], rotation = 60)

# plt.savefig("../figs/frequency-components-importance-complete.pdf")
plt.show()

#### Square masks

Replication of the frequency components importance plot with square masks [(Zhang et al, 2022)](https://www.sciencedirect.com/science/article/abs/pii/S0925231222001084)

In [ ]:
# perturbation 
perturbation = 'square' # other values are 'square' and 'grid' for the replication of 
                        # former paper's results 
grid_size = 14
results = {}

for case, model in zip(cases, models):
    print('Case ........... {}'.format(case))
    fourier = FourierSobol(model, grid_size = grid_size, nb_design = 16, batch_size=128, perturbation = perturbation)
    _ = fourier(x,y)
    results[case] = fourier.components

# plots
# represent the importance of each frequency component in each case-
plt.rcParams.update({'font.size': 12})

# average the contributions per training mode
average_baseline = np.sum(results['baseline'], axis = 0) / n_samples
average_robust = np.sum(
    [
    np.sum(results[case], axis = 0) / n_samples for case in ['augmix', 'pixmix', 'sin']
    ], axis = 0) / 3

average_adversarial = np.sum(
    [
    np.sum(results[case], axis = 0) / n_samples for case in ['adv_free', 'fast_adv', "adv"]
    ], axis = 0) / 3

# average the contributions for each case
average_contributions = {
    'ST' : average_baseline,
    'RT' : average_robust,
    'AT' : average_adversarial
    }

# plots
for i, case in enumerate(average_contributions.keys()):
    average_contribution = average_contributions[case]

    offset = 0.33 * i

    plt.bar(np.array([*range(len(average_contribution))]) - 0.33 + offset , average_contribution, label = case, width = 0.25)

plt.legend()

# plot labels
plt.xlabel('Frequency component\n (The higher the higher the frequency)')
plt.ylabel('Importance (STI)')
plt.title('Importance of the frequency components (square case)')
plt.xticks([*range(0, grid_size)], rotation = 60)

plt.savefig("../figs/frequency-components-importance-squares-complete.pdf")

plt.show()

#### Grid masks (visualization of the importance in the nyquist Square)

Replication of the same results but with grid masks [(Chen et al. 2022)](https://openreview.net/forum?id=rQ1cNbi07Vq)

In [ ]:
perturbation = 'grid'
grid_size = 17
results = {}

for case, model in zip(cases, models):
    print('Case ............. {}'.format(case))
    
    fourier = FourierSobol(model, grid_size = grid_size, nb_design = 8, batch_size=128, perturbation = perturbation)
    explanations = fourier(x,y)
    results[case] = explanations

In [ ]:
fig, ax = plt.subplots(1,7, figsize = (28, 4))
plt.rcParams.update({'font.size': 15})

for i, case in enumerate(list(results.keys())):

    mean = np.zeros((grid_size, grid_size))

    count = 0
    for map in results[case]:
        if not np.isnan(map).any():

            mean += cv2.resize(map, (grid_size, grid_size))
            count += 1
            
    mean /= count
    
    ax[i].set_title('{}'.format(case))
    im = ax[i].imshow(np.log(1 + mean), cmap = 'jet')

    ax[i].set_xticks(([*range(grid_size)]))
    ax[i].set_xticklabels(np.array([*range(grid_size)]) - (grid_size // 2))

    ax[i].set_yticks(([*range(grid_size)]))
    ax[i].set_yticklabels(np.array([*range(grid_size)]) - (grid_size // 2))

    fig.colorbar(im, ax = ax[i], shrink = 0.8)
plt.suptitle('Frequency importance in the Nyquist square for baseline, robust and adversarial training')
fig.tight_layout()

#plt.savefig('../figs/frequency-grid-complete.pdf')
plt.show()

### Consistency between WCAM and Fourier-CAM

We show that we have similar results as in the previous plot with the WCAM

In [ ]:
grid_size = 28
levels = 5
opt = {
    'approximation' : True,
    'size' : grid_size
}

results = {}

for case, model in zip(cases, models):
    print('Case ............... {}'.format(case))
    wavelet = WaveletSobol(model, grid_size = grid_size, nb_design = 8, batch_size=128, opt = opt, levels = levels)
    explanations = wavelet(x,y)
    results[case] = explanations

In [ ]:
# consider the means
levels = 5

means = np.zeros((len(results.keys()), grid_size, grid_size))

for i, case in enumerate(list(results.keys())):

    if case == "baseline":
        wcams = results[case][1:]
    else:
        wcams = results[case]

    for wcam in wcams:
        means[i] += cv2.resize(wcam, (grid_size, grid_size))

    means[i] /= n_samples # average the values
    means[0,0] = 0 # remove the 0th component (approximation coefficient)

# work on the explanations to recover the frequencies 
averaged_coefficients = [helpers.reshape_wcam(means[i], grid_size, levels = levels) for i in range(means.shape[0])]

# Reshape the contributions per ST, RT, AT
baseline_coefficients = averaged_coefficients[0]
robust_coefficients = np.mean(averaged_coefficients[1:4], axis = 0)
adversarial_coefficients = np.mean(averaged_coefficients[-3:], axis = 0)

# group again the coefficients
averaged_coefficients = [baseline_coefficients, robust_coefficients, adversarial_coefficients]

# plots
plt.rcParams.update({'font.size': 12})

group_cases = ['ST', 'RT', 'AT']

for i, (average_contribution, case) in enumerate(zip(averaged_coefficients, group_cases)):
    offset = 0.33 * i
    plt.bar(np.array([*range(len(average_contribution) - 1)]) - 0.33 + offset , average_contribution[1:] / sum(average_contribution[1:]), label = case, width = 0.2)

# labels = [['a  \n 0']]

labels = []

for level in range(levels):
    labels.append(['h', 'd \n{}'.format(level + 1), 'v'])
labels = list(sum(labels, []))

plt.xticks([*range(len(labels))], labels = labels)
plt.title('Importance of the frequency components (WCAM case) \n (without approximation coefficient)')
plt.xlabel('Frequency/level')
plt.ylabel('Importance (normalized STIs)')
plt.legend()
# plt.savefig('../figs/wcam-fcam-consistency-complete-no-approx.pdf')
plt.show()